In [28]:
ord("h")

104

In [29]:
list("hello bro".encode("utf-8"))

[104, 101, 108, 108, 111, 32, 98, 114, 111]

In [30]:
list("hello bro".encode("utf-16"))

[255,
 254,
 104,
 0,
 101,
 0,
 108,
 0,
 108,
 0,
 111,
 0,
 32,
 0,
 98,
 0,
 114,
 0,
 111,
 0]

In [31]:

text = "Computation as a Resource One view of deep learning, is that neural networks can be characterized by the amount of computation and storage they can access in a forward pass, and if we optimize them to solve problems using gradient descent, the optimization process will figure out how to use these resources–they’ll figure out how to organize these resources into circuits for calculation and information storage. From this view, if we design an architecture or system that can do more computation at test time, and we train it to effectively use this resource, it’ll work better. In Transformer models, the amount of computation (flops) that the model does for each generated token is roughly 2 times the number of parameters. For sparse models like mixture of experts (MoE), only a fraction of the parameters are used in each forward pass, so computation = 2 * parameters / sparsity, where sparsity is the fraction of experts active. On the other hand, CoT enables the model to perform far more flops of computation for each token of the answer that it is trying to compute. In fact, CoT has a nice property that it allows the model to use a variable amount of compute depending on the hardness of the problem. Branching and Editing The fundamental intent of test-time compute is to adaptively modify the model’s output distribution at test time. There are various ways of utilizing test time resources for decoding to select better samples and thus alter the model’s predictions towards a more desired distribution. Two main approaches for improving the decoding process are parallel sampling and sequential revision. Parallel sampling generates multiple outputs simultaneously, meanwhile providing guidance per step with process reward signals or using verifiers to judge the quality at the end. It is the most widely adopted decoding method to improve test time performance, such as best-of- or beam search. Self-consistency (Wang et al. 2023) is commonly used to select the answer with majority vote among multiple CoT rollouts when the ground truth is not available. Sequential revision adapts the model’s responses iteratively based on the output in the previous step, asking the model to intentionally reflect its existing response and correct mistakes. The revision process may have to rely on a fine-tuned model, as naively relying on the model’s intrinsic capability of self-correction without external feedback may not lead to improvement (Kamoi et al. 2024, Huang et al. 2024). Parallel sampling is simple, intuitive and easier to implement, but bounded by the model capability of whether it can achieve the correct solution in one-go. Sequential explicitly asks the model to reflect on mistakes but it is slower and requires extra care during implementation as it does run the risk of correct predictions being modified to be incorrect or introducing other types of hallucinations. These two methods can be used together. Snell et al. (2024) showed that easier questions benefit from purely sequential test-time compute, whereas harder questions often perform best with an optimal ratio of sequential to parallel compute."
tokens = text.encode("utf-8")
tokens  = list(map(int, tokens))
print(len(tokens))
print(tokens)


3150
[67, 111, 109, 112, 117, 116, 97, 116, 105, 111, 110, 32, 97, 115, 32, 97, 32, 82, 101, 115, 111, 117, 114, 99, 101, 32, 79, 110, 101, 32, 118, 105, 101, 119, 32, 111, 102, 32, 100, 101, 101, 112, 32, 108, 101, 97, 114, 110, 105, 110, 103, 44, 32, 105, 115, 32, 116, 104, 97, 116, 32, 110, 101, 117, 114, 97, 108, 32, 110, 101, 116, 119, 111, 114, 107, 115, 32, 99, 97, 110, 32, 98, 101, 32, 99, 104, 97, 114, 97, 99, 116, 101, 114, 105, 122, 101, 100, 32, 98, 121, 32, 116, 104, 101, 32, 97, 109, 111, 117, 110, 116, 32, 111, 102, 32, 99, 111, 109, 112, 117, 116, 97, 116, 105, 111, 110, 32, 97, 110, 100, 32, 115, 116, 111, 114, 97, 103, 101, 32, 116, 104, 101, 121, 32, 99, 97, 110, 32, 97, 99, 99, 101, 115, 115, 32, 105, 110, 32, 97, 32, 102, 111, 114, 119, 97, 114, 100, 32, 112, 97, 115, 115, 44, 32, 97, 110, 100, 32, 105, 102, 32, 119, 101, 32, 111, 112, 116, 105, 109, 105, 122, 101, 32, 116, 104, 101, 109, 32, 116, 111, 32, 115, 111, 108, 118, 101, 32, 112, 114, 111, 98, 108, 101, 1

In [32]:

def get_stats(ids):
    counts = {}
    for pair in zip(ids, ids[1:]):
        counts[pair] = counts.get(pair, 0) + 1 
    return counts

stats = get_stats(tokens)
print(stats)
print(sorted(((v,k) for k,v in stats.items()), reverse=True))

{(67, 111): 4, (111, 109): 14, (109, 112): 21, (112, 117): 15, (117, 116): 28, (116, 97): 11, (97, 116): 24, (116, 105): 48, (105, 111): 28, (111, 110): 37, (110, 32): 44, (32, 97): 52, (97, 115): 13, (115, 32): 68, (97, 32): 8, (32, 82): 1, (82, 101): 1, (101, 115): 44, (115, 111): 8, (111, 117): 21, (117, 114): 12, (114, 99): 8, (99, 101): 14, (101, 32): 84, (32, 79): 2, (79, 110): 2, (110, 101): 12, (32, 118): 6, (118, 105): 8, (105, 101): 8, (101, 119): 3, (119, 32): 3, (32, 111): 41, (111, 102): 20, (102, 32): 20, (32, 100): 14, (100, 101): 24, (101, 101): 2, (101, 112): 4, (112, 32): 2, (32, 108): 3, (108, 101): 22, (101, 97): 11, (97, 114): 27, (114, 110): 2, (110, 105): 3, (105, 110): 39, (110, 103): 25, (103, 44): 1, (44, 32): 20, (32, 105): 38, (105, 115): 22, (32, 116): 82, (116, 104): 55, (104, 97): 13, (116, 32): 50, (32, 110): 7, (101, 117): 1, (114, 97): 22, (97, 108): 24, (108, 32): 23, (101, 116): 14, (116, 119): 2, (119, 111): 4, (111, 114): 31, (114, 107): 2, (107, 1

In [33]:
top_pair = max(stats, key=stats.get)
top_pair

(101, 32)

In [34]:

def merge(ids, pair, idx):
  newids = []
  i = 0
  while i < len(ids):
    if i < len(ids) - 1 and ids[i] == pair[0] and ids[i+1] == pair[1]:
      newids.append(idx)
      i += 2
    else:
      newids.append(ids[i])
      i += 1
  return newids

print(merge([5, 6, 6, 7, 9, 1], (6, 7), 99))

tokens2 = merge(tokens, top_pair, 256)
print(tokens2)
print("length:", len(tokens2))


[5, 6, 99, 9, 1]
[67, 111, 109, 112, 117, 116, 97, 116, 105, 111, 110, 32, 97, 115, 32, 97, 32, 82, 101, 115, 111, 117, 114, 99, 256, 79, 110, 256, 118, 105, 101, 119, 32, 111, 102, 32, 100, 101, 101, 112, 32, 108, 101, 97, 114, 110, 105, 110, 103, 44, 32, 105, 115, 32, 116, 104, 97, 116, 32, 110, 101, 117, 114, 97, 108, 32, 110, 101, 116, 119, 111, 114, 107, 115, 32, 99, 97, 110, 32, 98, 256, 99, 104, 97, 114, 97, 99, 116, 101, 114, 105, 122, 101, 100, 32, 98, 121, 32, 116, 104, 256, 97, 109, 111, 117, 110, 116, 32, 111, 102, 32, 99, 111, 109, 112, 117, 116, 97, 116, 105, 111, 110, 32, 97, 110, 100, 32, 115, 116, 111, 114, 97, 103, 256, 116, 104, 101, 121, 32, 99, 97, 110, 32, 97, 99, 99, 101, 115, 115, 32, 105, 110, 32, 97, 32, 102, 111, 114, 119, 97, 114, 100, 32, 112, 97, 115, 115, 44, 32, 97, 110, 100, 32, 105, 102, 32, 119, 256, 111, 112, 116, 105, 109, 105, 122, 256, 116, 104, 101, 109, 32, 116, 111, 32, 115, 111, 108, 118, 256, 112, 114, 111, 98, 108, 101, 109, 115, 32, 117, 11

In [35]:
def get_stats(ids):
    counts = {}
    for pair in zip(ids, ids[1:]):
        counts[pair] = counts.get(pair, 0) +1
    return counts

def merge(ids, pair, idx):
    newids =[]
    i = 0
    while i < len(ids):
        if i < len(ids) - 1 and ids[i] == pair[0] and ids[i+1] == pair[1]:
            newids.append(idx)
            i +=2
        else: 
            newids.append(ids[i])
            i += 1
    return newids

vocab_size = 276
num_merges = vocab_size - 256
ids = list(tokens)

merges = {}
for i in range(num_merges):
    stats = get_stats(ids)
    pair = max(stats, key=stats.get)
    idx = 256 + i
    print(f"mergingg {pair} into a new token {idx}")
    ids = merge(ids, pair, idx)
    merges[pair] = idx




mergingg (101, 32) into a new token 256
mergingg (32, 116) into a new token 257
mergingg (115, 32) into a new token 258
mergingg (116, 105) into a new token 259
mergingg (116, 32) into a new token 260
mergingg (111, 110) into a new token 261
mergingg (105, 110) into a new token 262
mergingg (257, 104) into a new token 263
mergingg (101, 114) into a new token 264
mergingg (101, 115) into a new token 265
mergingg (111, 114) into a new token 266
mergingg (101, 108) into a new token 267
mergingg (97, 110) into a new token 268
mergingg (263, 256) into a new token 269
mergingg (97, 114) into a new token 270
mergingg (111, 32) into a new token 271
mergingg (97, 108) into a new token 272
mergingg (101, 110) into a new token 273
mergingg (259, 261) into a new token 274
mergingg (109, 112) into a new token 275


In [36]:
print("tokjen length : ", len(tokens))
print("ids length", len(ids))
print(f"compression ratio {len(tokens)/len(ids):.2f}X")


tokjen length :  3150
ids length 2417
compression ratio 1.30X


In [ ]:

vocab = { idx: bytes([idx]) for idx in range(256)}
for (p0, p1), idx in merges.items():
    vocab[idx] = vocab[p0] + vocab[p1]

def decode(ids):
    tokens = b"".join(vocab[idx] for idx in ids)
    text = tokens.decode("utf-8", errors="replace")
    return text

print(decode([129]))


�


In [49]:
def encode(text):
    tokens = list(text.encode("utf-8"))
    while len(tokens)>=2:
        stats = get_stats(tokens)
        pair = min(stats, key=lambda p: merges.get(p, float("inf")))
        if pair not in merges: 
            break 
        idx = merges[pair]
        tokens = merge(tokens, pair, idx)
    return tokens

print(encode("hello brother"))

[104, 267, 108, 271, 98, 114, 111, 116, 104, 264]


gpt series 

In [53]:
import regex as re 
gpt2pat = re.compile(r"""'s|'t|'re|'ve|'m|'ll|'d| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+""")

text = "Hello, I'm learning NLP in 2026!"
tokens = gpt2pat.findall(text)
tokens

['Hello', ',', ' I', "'m", ' learning', ' NLP', ' in', ' 2026', '!']

In [54]:
import tiktoken
enc = tiktoken.get_encoding("gpt2")
print(enc.encode("       hellllooo brother again !!!! "))

enc = tiktoken.get_encoding("cl100k_base")
print(enc.encode("       hellllooo brother again !!!! "))



/Users/dilshodbekikromov/Desktop/CMU Advanced NLP/tokeniser/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


[220, 220, 220, 220, 220, 220, 5968, 297, 34160, 3956, 757, 220, 13896, 220]
[996, 15123, 75, 385, 2689, 10868, 1578, 758, 12340, 220]


In [58]:
import sentencepiece as spm 



In [59]:
with open("toy.txt", "w", encoding="utf-8") as f:
  f.write("SentencePiece is an unsupervised text tokenizer and detokenizer mainly for Neural Network-based text generation systems where the vocabulary size is predetermined prior to the neural model training. SentencePiece implements subword units (e.g., byte-pair-encoding (BPE) [Sennrich et al.]) and unigram language model [Kudo.]) with the extension of direct training from raw sentences. SentencePiece allows us to make a purely end-to-end system that does not depend on language-specific pre/postprocessing.")

In [60]:
import os

options = dict(
  input="toy.txt",
  input_format="text",
  model_prefix="tok400",
  model_type="bpe",
  vocab_size=400,
  normalization_rule_name="identity",
  remove_extra_whitespaces=False,
  input_sentence_size=200000000,
  max_sentence_length=4192,
  seed_sentencepiece_size=1000000,
  shuffle_input_sentence=True,
  character_coverage=0.99995,
  byte_fallback=True,
  split_digits=True,
  split_by_unicode_script=True,
  split_by_whitespace=True,
  split_by_number=True,
  max_sentencepiece_length=16,
  add_dummy_prefix=True,
  allow_whitespace_only_pieces=True,
  unk_id=0,
  bos_id=1,
  eos_id=2,
  pad_id=-1,
  num_threads=os.cpu_count(),
)

spm.SentencePieceTrainer.train(**options)


sentencepiece_trainer.cc(78) LOG(INFO) Starts training with : 
trainer_spec {
  input: toy.txt
  input_format: text
  model_prefix: tok400
  model_type: BPE
  vocab_size: 400
  self_test_sample_size: 0
  character_coverage: 0.99995
  input_sentence_size: 200000000
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 10
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 1
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 1
  required_chars: 
  byte_fallback: 1
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  seed_sentencepieces_file: 
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 0
  bos_id: 1
  eos_id: 2
  pad_id: -1
  unk_piece: <unk>
  bos_piece: <s>
  eos_piece: </s>
  pad_piece: <pad>
  unk_surface:  ⁇ 
  enable_differential_privacy: 0
  diff

In [61]:
sp = spm.SentencePieceProcessor()
sp.load('tok400.model')
vocab = [[sp.id_to_piece(idx), idx] for idx in range(sp.get_piece_size())]
vocab

[['<unk>', 0],
 ['<s>', 1],
 ['</s>', 2],
 ['<0x00>', 3],
 ['<0x01>', 4],
 ['<0x02>', 5],
 ['<0x03>', 6],
 ['<0x04>', 7],
 ['<0x05>', 8],
 ['<0x06>', 9],
 ['<0x07>', 10],
 ['<0x08>', 11],
 ['<0x09>', 12],
 ['<0x0A>', 13],
 ['<0x0B>', 14],
 ['<0x0C>', 15],
 ['<0x0D>', 16],
 ['<0x0E>', 17],
 ['<0x0F>', 18],
 ['<0x10>', 19],
 ['<0x11>', 20],
 ['<0x12>', 21],
 ['<0x13>', 22],
 ['<0x14>', 23],
 ['<0x15>', 24],
 ['<0x16>', 25],
 ['<0x17>', 26],
 ['<0x18>', 27],
 ['<0x19>', 28],
 ['<0x1A>', 29],
 ['<0x1B>', 30],
 ['<0x1C>', 31],
 ['<0x1D>', 32],
 ['<0x1E>', 33],
 ['<0x1F>', 34],
 ['<0x20>', 35],
 ['<0x21>', 36],
 ['<0x22>', 37],
 ['<0x23>', 38],
 ['<0x24>', 39],
 ['<0x25>', 40],
 ['<0x26>', 41],
 ['<0x27>', 42],
 ['<0x28>', 43],
 ['<0x29>', 44],
 ['<0x2A>', 45],
 ['<0x2B>', 46],
 ['<0x2C>', 47],
 ['<0x2D>', 48],
 ['<0x2E>', 49],
 ['<0x2F>', 50],
 ['<0x30>', 51],
 ['<0x31>', 52],
 ['<0x32>', 53],
 ['<0x33>', 54],
 ['<0x34>', 55],
 ['<0x35>', 56],
 ['<0x36>', 57],
 ['<0x37>', 58],
 ['<0x38>', 5

In [63]:
ids = sp.encode("My chineese name is 迪硕 and it is pronouced as Dí Shuò")
print(ids)

[362, 80, 381, 362, 371, 378, 262, 361, 352, 361, 312, 367, 375, 361, 326, 362, 235, 194, 173, 234, 164, 152, 332, 362, 306, 326, 291, 284, 368, 373, 261, 370, 264, 369, 362, 71, 198, 176, 286, 378, 373, 198, 181]


In [64]:
print([sp.id_to_piece(idx) for idx in ids])

['▁', '<0x4D>', 'y', '▁', 'c', 'h', 'in', 'e', 'es', 'e', '▁n', 'a', 'm', 'e', '▁is', '▁', '<0xE8>', '<0xBF>', '<0xAA>', '<0xE7>', '<0xA1>', '<0x95>', '▁and', '▁', 'it', '▁is', '▁pr', 'on', 'o', 'u', 'ce', 'd', '▁a', 's', '▁', '<0x44>', '<0xC3>', '<0xAD>', '▁S', 'h', 'u', '<0xC3>', '<0xB2>']
